# Capstone Project Pipeline

## Preprocessing

In [1]:
import glob
import csv
import numpy as np

In [2]:
def extract_box(path):
    """extract_box
    Extract annotation box positions for each labels from VIVA hand dataset.
    output is a list of tuples.

    :param path: text file path
    """

    with open(path) as temp:
        output = []

        for i, line in enumerate(temp):

            if i != 0 and line:
                label, x_1, y_1, x_off, y_off, *_ = line.split()
                pt_1 = (int(x_1), int(y_1))
                pt_2 = (pt_1[0] + int(x_off), (pt_1[1] + int(y_off)))
                output.append((label, pt_1, pt_2))

    return output

def create_csv(image_dir, annotation_dir, csv_out_path, val_out_path=None, val_split=None):
    image_paths = sorted(glob.glob(image_dir + '*'))
    annotations_paths = sorted(glob.glob(annotation_dir + '*'))

    # each image can have up to 4 hand bboxes
    rows = []
    for image_path, annotations_path in zip(image_paths, annotations_paths):
            annotations = extract_box(annotations_path)
            for annotation in annotations:
                # annotation [label, (x1, y1), (x2, y2)]
                # save as image,x1,y2,x2,y2,label
                rows.append([image_path,
                             annotation[1][0], annotation[1][1],
                             annotation[2][0], annotation[2][1],
                             annotation[0]])
    if val_split:
        # shuffle and split
        np.random.shuffle(rows)
        val_size = int(len(rows) * val_split)
        val_rows = rows[:val_size]
        with open('./validation.csv' if val_out_path is None else val_out_path, 'w') as csv_file:
            writer = csv.writer(csv_file)
            for row in val_rows:
                writer.writerow(row)
        rows = rows[val_size:]

    with open(csv_out_path, 'w') as csv_file:
            writer = csv.writer(csv_file)
            for row in rows:
                writer.writerow(row)

In [3]:
# this is the root directory where the training data is extracted
data_dir = '/media/appsyoon/New Volume/Machine Learning/data/'
# training data path
train_dir = data_dir + 'detectiondata/train/'
train_image_dir = train_dir + 'pos/'
train_annotation_dir = train_dir + 'posGt/'

out_path = './train.csv'

create_csv(train_image_dir, train_annotation_dir, out_path, val_split=0.2)

In [4]:
# the test data images are in the same root dir as training
test_image_dir = data_dir + 'detectiondata/test/pos/'
# but the annotations are downloaded separately and extracted into data_dir/evaluation/
test_annotation_dir = data_dir + 'evaluation/annotations/'

test_out_path = './test.csv'

create_csv(test_image_dir, test_annotation_dir, test_out_path)

## Training

In [10]:
import keras

from keras_resnet.models import ResNet50

from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.models.resnet import WEIGHTS_PATH_NO_TOP_50
from keras_retinanet import losses
from keras_retinanet.models.retinanet import retinanet_bbox

In [11]:
batch_size = 1
epochs = 2

In [12]:
def create_generator():
    # horizontal flip for preprocessing augmentation
    train_generator = ImageDataGenerator(horizontal_flip=True)
    # wrap in CSVGenerators
    csv_train_generator = CSVGenerator('./train.csv', './classes.csv', 
                                       train_generator, batch_size=batch_size)
    # no flip for val
    val_generator = ImageDataGenerator()
    csv_val_generator = CSVGenerator('./train.csv', './classes.csv', 
                                       val_generator, batch_size=batch_size)
    return csv_train_generator, csv_val_generator

In [13]:
def create_model(num_classes):
    input_shape = keras.layers.Input((None, None, 3))
    
    # first create the backbone ResNet50 model
    weights_path = keras.applications.imagenet_utils.get_file('ResNet-50-model.keras.h5', WEIGHTS_PATH_NO_TOP_50, 
                                                              cache_subdir='models', md5_hash='3e9f4e4f77bbe2c9bec13b53ee1c2319')
    backbone_model = ResNet50(input_shape, include_top=False, freeze_bn=True)
    # add feature pyramid network and subnets for box regression and object classification
    model = retinanet_bbox(inputs=input_shape, num_classes=num_classes, backbone=backbone_model)
    model.load_weights(weights_path, by_name=True)
    # compile model
    model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )
    
    return model

In [14]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [15]:
def create_callbacks():
    checkpoint = keras.callbacks.ModelCheckpoint('resnet50_csv_best.h5', verbose=1)
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)
    loss_history = LossHistory()
    
    return [checkpoint, lr_scheduler, loss_history]

In [16]:
train_generator, val_generator = create_generator()
model = create_model(train_generator.num_classes())
print(model.summary())

callbacks = create_callbacks()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu

/home/appsyoon/miniconda3/envs/capstone/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Output "nms" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "nms" during training.


In [17]:
history = model.fit_generator(generator=train_generator, steps_per_epoch=1, epochs=epochs, 
                              validation_data=val_generator, validation_steps=1, 
                              callbacks=callbacks, verbose=1)

Epoch 1/2
Epoch 00001: saving model to resnet50_csv_best.h5
1/1 [==============================] - 22s 22s/step - loss: 5.3324 - regression_loss: 4.2012 - classification_loss: 1.1312 - val_loss: 5.5361 - val_regression_loss: 4.4033 - val_classification_loss: 1.1328
Epoch 2/2
Epoch 00002: saving model to resnet50_csv_best.h5
1/1 [==============================] - 17s 17s/step - loss: 5.4562 - regression_loss: 4.3227 - classification_loss: 1.1335 - val_loss: 5.6330 - val_regression_loss: 4.4990 - val_classification_loss: 1.1340


In [25]:
# save the history as a json for chart
import pickle

with open('history.pkl' , 'wb') as history_file:
    pickle.dump(history.history, history_file)

In [26]:
with open('history.pkl', 'rb') as history_file:
    data = pickle.load(history_file)
    print(data)

{'val_loss': [5.5361065864562988, 5.633028507232666], 'val_regression_loss': [4.4032983779907227, 4.4989986419677734], 'val_classification_loss': [1.1328083276748657, 1.1340299844741821], 'loss': [5.3323974609375, 5.4561867713928223], 'regression_loss': [4.201179027557373, 4.3226919174194336], 'classification_loss': [1.1312186717987061, 1.1334948539733887], 'lr': [9.9999997e-06, 9.9999997e-06]}
